In [1]:
import gym
import torch
from myrl.buffers import ReplayBuffer
from myrl.visualizer import showit
from myrl.environments import Envs
from myrl.policies import RandomPolicy
from myrl.model_based import ModelResidual

import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from myrl.utils import get_batch_a, get_batch_obs

envname = 'BipedalWalker-v3'
# envname = 'LunarLanderContinuous-v2'
envname = 'Pendulum-v0'
env = gym.make(envname)
envs = Envs(envname, 1)

In [9]:
rbuff = ReplayBuffer(1000000)

In [17]:
rdim = 1
adim = env.action_space.shape[0]
sdim = env.observation_space.shape[0]
idim = adim + sdim
odim = rdim + sdim
model = ModelResidual([idim, 128, 64, sdim], env)
random_policy = RandomPolicy(env).act
obstest = get_batch_obs(env, 1)
adim, sdim

(1, 3)

In [16]:
opt = torch.optim.Adam(model.parameters(), lr=5e-2)

In [14]:
# exname = 'model_'
# os.system('mkdir tb/' + exname)
# writer = SummaryWriter('tb/'+exname)

bsize = 128
loss = 0; rloss=0; oloss=0;
for ep in range(10000):
    obs = env.reset()
    if ep%1==0:
        for step in range(1):
            oldobs, a, r, obs, d, _, _, _ = envs.rollout(random_policy)
            repbuff.add(oldobs, a, r, obs, d)
    if bsize*3 > len(repbuff):
        print(len(rbuff), end='\r')
        continue
    oldobs, a, r, obs, d = repbuff.get(bsize)

    for opt_step in range(3):
        pr, pobs = model(oldobs, a)
        rloss = ((pr-r)**2).mean()
        oloss = ((pobs-obs)**2).mean()
        loss = rloss + 10*torch.abs(rloss/oloss).detach()*oloss
        opt.zero_grad()
        loss.backward()
        opt.step()
    # writer.add_scalar('loss/model', loss.item(), ep)
    # writer.add_scalar('loss/reward', rloss.item(), ep)
    # writer.add_scalar('loss/states', oloss.item(), ep)
    
    if ep%10==0:
        print(ep, loss.item(),rloss.item(), oloss.item())

ModelResidual(
  (layers): ModuleList(
    (0): Linear(in_features=4, out_features=128, bias=True)
    (1): Linear(in_features=128, out_features=64, bias=True)
    (2): Linear(in_features=64, out_features=3, bias=True)
    (3): Linear(in_features=64, out_features=1, bias=True)
  )
)
0 1.89307701587677 0.17209790647029877 8.687705121701583e-05
10 1.348448395729065 0.1225862205028534 8.688236266607419e-05
20 1.2887029647827148 0.1171548068523407 5.5397063988493755e-05
30 1.9946781396865845 0.18133437633514404 0.0001424619840690866
40 1.6623473167419434 0.15112249553203583 7.165518763940781e-05
50 1.9079346656799316 0.1734486073255539 0.0001156237194663845
60 1.5558867454528809 0.14144425094127655 0.00012447040353436023
70 1.7602776288986206 0.1600252389907837 7.561735401395708e-05
80 1.3217402696609497 0.12015821784734726 9.938963921740651e-05
90 1.4263582229614258 0.1296689361333847 0.0003364376025274396
100 1.1327593326568604 0.10297811776399612 6.915862468304113e-05
110 1.426415085792

KeyboardInterrupt: 

In [15]:
entacter = lambda obs: (model.cross_entropy(torch.tensor(obs).float(), 3, 60, 25, pmax=0.15), -1)
gradactor= lambda obs: (model.gradient_optimize(obs, 20, [-2, 2], grad_step=1), -1)
showit(env, gradactor, max_steps=200, time_delta=0)

-411.02996878073657